In [ ]:
!pip install numpy
!pip install scipy
!pip install pandas
!pip install pathlib
!pip install zipfile
!pip install tensorboard
!pip install tensorflow
!pip install tensorflow-estimator
!pip install scikit-learn
!pip install scikit-image
!pip install scikit-fuzzy
!pip install matplotlib
!pip install -U hopsworks --quiet
!pip install joblib

ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894077 sha256=21d44d22169f64bc619ec1c573f243017dbef35b7d9ee538888b9d8d9282a3d3
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.8/249.8 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 5.6 MB/

In [ ]:
%%writefile autoencoder.py
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from scipy.fft import rfft, rfftfreq
from scipy import signal
import sklearn
import os

# AutoEncoder architecture

input_freq = keras.Input(shape=(641,))
# "encoded" is the encoded representation of the input
encoded = keras.layers.Dense(256, activation="relu")(input_freq)
encoded = keras.layers.Dense(128, activation="relu")(encoded)
encoded = keras.layers.Dense(64, activation="relu")(encoded)
encoded = keras.layers.Dense(32, activation="relu")(encoded)
encoded = keras.layers.Dense(8, activation="relu")(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = keras.layers.Dense(8, activation="relu")(encoded)
decoded = keras.layers.Dense(32, activation="relu")(decoded)
decoded = keras.layers.Dense(64, activation="relu")(decoded)
decoded = keras.layers.Dense(128, activation="relu")(decoded)
decoded = keras.layers.Dense(641, activation="linear")(decoded)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input_freq, decoded)
# This model maps an input to its encoded representation
encoder = keras.Model(input_freq, encoded)
weights = autoencoder.get_weights()
autoencoder.compile(optimizer="adam", loss="mae")


def anomaly_score(X_true, X_pred):

    if X_true.shape != X_pred.shape:
        raise ValueError(
            f"arrays must be of the same shape. X_true is {X_true.shape} while X_pred is {X_pred.shape}"
        )

    score = np.abs(X_true - X_pred).mean(axis=1)

    return score


def calc_anomaly_treshold(x):

    threshold = np.mean(x) + 3 * np.std(x)

    return threshold


def correct_labels(Hl, Al):
    labelsb = Al
    labelsb = np.where(labelsb == 1, 3, 2)
    avg = []
    for i in range(3):
        avg.append(np.sum(np.where(Hl == i)) / len(np.where(Hl == i)[0]))
    new = np.copy(Hl)
    for i in range(3):
        idx = np.argmin(avg)
        new[np.where(Hl == idx)] = i
        avg[idx] = float("inf")
    labelsb[: Hl.shape[0]] = new
    return labelsb


def get_AElabels(Xt, Xf):
    autoencoder.set_weights(weights)
    autoencoder.fit(
        Xt[:, :641], Xt[:, :641], epochs=100, batch_size=64, shuffle=True, verbose=False
    )

    # Horizontal vibration encoding
    Hencoded_f = encoder.predict(Xt[:, :641])

    Hdecoded_f = autoencoder.predict(Xf[:, :641])
    score = anomaly_score(Xf[:, :641], Hdecoded_f)
    anomaly_threshold = calc_anomaly_treshold(
        anomaly_score(Xt[:, :641], autoencoder.predict(Xt[:, :641]))
    )

    autoencoder.set_weights(weights)
    autoencoder.fit(
        Xt[:, 641:], Xt[:, 641:], epochs=100, batch_size=64, shuffle=True, verbose=False
    )

    # Vertical vibration encoding
    Vencoded_f = encoder.predict(Xt[:, 641:])

    Encoding = np.concatenate((Hencoded_f, Vencoded_f), axis=1)
    kmeans = KMeans(n_clusters=3, random_state=0).fit(Encoding)
    kMeanslabels = kmeans.labels_

    Alabels = np.where(score > anomaly_threshold, 1, 0)

    return correct_labels(kMeanslabels, Alabels)

Writing autoencoder.py


In [ ]:
%%writefile NNclassifier.py
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from random import randint

# classifier architecture


def create_model():

    H_input = keras.Input(shape=(641,), name="horizontal")
    V_input = keras.Input(shape=(641,), name="vertical")
    meta_input = keras.Input(shape=(26,), name="meta_input")
    initializer = tf.keras.initializers.RandomNormal(seed=42)

    H = keras.layers.Dense(
        256, activation="relu", kernel_initializer=initializer, kernel_regularizer="l2"
    )(H_input)
    H = keras.layers.Dense(
        128, activation="relu", kernel_initializer=initializer, kernel_regularizer="l2"
    )(H)
    H = keras.layers.Dense(
        64, activation="relu", kernel_initializer=initializer, kernel_regularizer="l2"
    )(H)
    H = keras.layers.Dense(
        32, activation="relu", kernel_initializer=initializer, kernel_regularizer="l2"
    )(H)
    H = keras.layers.Dense(4, activation="relu", kernel_initializer=initializer)(H)

    V = keras.layers.Dense(
        256, activation="relu", kernel_initializer=initializer, kernel_regularizer="l2"
    )(V_input)
    V = keras.layers.Dense(
        128, activation="relu", kernel_initializer=initializer, kernel_regularizer="l2"
    )(V)
    V = keras.layers.Dense(
        64, activation="relu", kernel_initializer=initializer, kernel_regularizer="l2"
    )(V)
    V = keras.layers.Dense(
        32, activation="relu", kernel_initializer=initializer, kernel_regularizer="l2"
    )(V)
    V = keras.layers.Dense(4, activation="relu", kernel_initializer=initializer)(V)

    F = keras.layers.Dense(16, activation="relu", kernel_initializer=initializer)(
        meta_input
    )
    F = keras.layers.Dense(4, activation="relu", kernel_initializer=initializer)(F)

    final = keras.layers.concatenate([H, V, F])
    final = keras.layers.Dense(
        64, activation="relu", kernel_initializer=initializer, kernel_regularizer="l2"
    )(final)
    final = keras.layers.Dense(
        32, activation="relu", kernel_initializer=initializer, kernel_regularizer="l2"
    )(final)
    final = keras.layers.Dense(4, activation="softmax", kernel_initializer=initializer)(
        final
    )
    model = keras.models.Model(inputs=[H_input, V_input, meta_input], outputs=[final])

    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

    return model

Writing NNclassifier.py


In [ ]:
import hopsworks
import numpy as np
import pandas as pd
from pathlib import Path
from zipfile import ZipFile
import os
import time
from scipy.fft import rfft, rfftfreq
from scipy.stats import kurtosis
import scipy.stats as stats
from scipy.signal import find_peaks
from scipy import signal

In [ ]:
project = hopsworks.login(api_key_value="<your api key>")
fs = project.get_feature_store()


# I stored test set from femto dataset in an S3 bucket.  You can think of an alternate storage
!wget --no-check-certificate --no-proxy "https://your-s3-bucket/Test_set.zip"

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/895126
Connected. Call `.close()` to terminate connection gracefully.
--2024-07-27 07:23:32--  https://bearingpred.s3.ap-southeast-1.amazonaws.com/Test_set.zip
Resolving bearingpred.s3.ap-southeast-1.amazonaws.com (bearingpred.s3.ap-southeast-1.amazonaws.com)... 52.219.125.86, 3.5.150.151, 3.5.148.27, ...
Connecting to bearingpred.s3.ap-southeast-1.amazonaws.com (bearingpred.s3.ap-southeast-1.amazonaws.com)|52.219.125.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 273416434 (261M) [application/zip]
Saving to: ‘Test_set.zip’

Test_set.zip        100%[===================>] 260.75M  18.6MB/s    in 16s     

2024-07-27 07:23:49 (16.6 MB/s) - ‘Test_set.zip’ saved [273416434/273416434]



In [ ]:
COLUMNS = ["Time", "Hacc", "Vacc", "Bearing"]

# change directory to raw files here and merged files at the end
#ROOT_FROM = Path("/content/training_files")
ROOT_TO = Path("/content/merged_files")

#ROOT_FROM.mkdir(parents=True, exist_ok=True)
ROOT_TO.mkdir(parents=True, exist_ok=True)

def read_bearings_zip(input_zip):
    """
    Extract the bearings Test_set.zip file into memory.

    Parameters
    ----------
    input_zip : Path or str
        Path to the Test_set.zip file.

    Returns
    -------
    files : dict of dicts
        Dict of dicts of contained .csv files as pandas dataframes.
    """

    input_zip = ZipFile(input_zip)
    files = {}
    for name in sorted(input_zip.namelist()):
        # Add 'directory' in dict if not yet present
        parent = Path(name).parent.stem
        if parent not in files:
            if "Bearing" not in parent:
                continue
            files[parent] = {}

        # Read and store .csv contents
        with input_zip.open(name) as file:
            if "acc" in name:
                files[parent][name] = pd.read_csv(
                    file,
                    header=None,
                    names=["h", "m", "s", "ms", "Hacc", "Vacc"],
                    dtype={"ms": np.int32},
                    sep=";"
                    if "1.4" in parent
                    else ",",  # 1.4 uses ; to separate values
                )
    return files

    # Read the zip file into memory
input_data = read_bearings_zip(Path("/content/Test_set.zip"))


for dir, files in input_data.items():
    print("Directory:", dir)

    # Concatenate individual .csv files
    tdf = pd.concat(files.values(), ignore_index=True)

    # Convert individual time columns to a single time column
    tdf["Time"] = pd.to_datetime(
        tdf["h"] * 3600000 + tdf["m"] * 60000 + tdf["s"] * 1000 + tdf["ms"] / 1000,
        unit="ms",
    ).dt.time
    tdf["Bearing"] = dir

    # Save to csv (slow)
    tdf[COLUMNS].to_csv(ROOT_TO / f"{dir}.csv", index=False)

Directory: Bearing1_3
Directory: Bearing1_4
Directory: Bearing1_5
Directory: Bearing1_6
Directory: Bearing1_7
Directory: Bearing2_3
Directory: Bearing2_4
Directory: Bearing2_5
Directory: Bearing2_6
Directory: Bearing2_7
Directory: Bearing3_3


In [ ]:
def ToFrequency(df):
    X = []

    for i in range(0, len(df), 2560):
        Hshort = signal.decimate(df["Hacc"][i : i + 2560], 2)
        Vshort = signal.decimate(df["Vacc"][i : i + 2560], 2)
        Hf = np.abs(rfft(Hshort))
        Vf = np.abs(rfft(Vshort))
        x = np.append(Hf, Vf)
        X.append(x)

    return X

result = []

rootdir = "/content/merged_files/"
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        df = pd.read_csv(rootdir + "/" + file)
        x = ToFrequency(df)
        result.extend(x)

df_freq = pd.DataFrame(result)
headers = []
for i in range(0, 641):
    headers.append("Hfreq_" + str(i))
for i in range(0, 641):
    headers.append("Vfreq_" + str(i))

df_freq.columns = headers
df_freq.to_csv("/content/Xtrain_freq.csv", index=False)


In [ ]:
def ToTime(df):

    Hzerocross = []
    Vzerocross = []
    Hkurtosis = []
    Vkurtosis = []
    Hrms = []
    Vrms = []
    Hpeaks = []
    Vpeaks = []
    Hmean = []
    Vmean = []
    Hstd = []
    Vstd = []
    Hmedian = []
    Vmedian = []
    Hskewness = []
    Vskewness = []
    Henergy = []
    Venergy = []
    Hshapiro = []
    Vshapiro = []
    Hkl = []
    Vkl = []
    Hrkl = []
    Vrkl = []
    Hcrest = []
    Vcrest = []

    for i in range(0, len(df), 2560):
        Horizontal = signal.decimate(df["Hacc"][i : i + 2560], 2)
        Vertical = signal.decimate(df["Vacc"][i : i + 2560], 2)

        # zero crossing
        Hc = ((Horizontal[:-1] * Horizontal[1:]) < 0).sum() + (Horizontal == 0).sum()
        Hzerocross.append(Hc)
        Vc = ((Vertical[:-1] * Vertical[1:]) < 0).sum() + (Vertical == 0).sum()
        Vzerocross.append(Vc)

        # kurtosis: distribution tail
        Hkur = kurtosis(Horizontal)
        Hkurtosis.append(Hkur)
        Vkur = kurtosis(Vertical)
        Vkurtosis.append(Vkur)

        # RMS  root-mean-square value of a signal: Loosely speaking, it represents the average "power" of a signal.
        # The root mean square (RMS) is widely used to represent HIs and is further employed to identify the initial
        # point of degradation when the RMS exceeds a pre-specified threshold
        # Wang, W. A model to predict the residual life of rolling element bearings given monitored condition information
        # to date. IMA J. Manag. Math. 2002, 13, 3–16.
        hrms = np.sqrt(np.mean(Horizontal**2))
        Hrms.append(hrms)
        vrms = np.sqrt(np.mean(Vertical**2))
        Vrms.append(vrms)

        # Number of peaks
        hpeaks, _ = find_peaks(Horizontal)
        Hpeaks.append(len(hpeaks) / len(Horizontal))
        vpeaks, _ = find_peaks(Vertical)
        Vpeaks.append(len(vpeaks) / len(Vertical))

        # Mean
        Hmean.append(np.mean(Horizontal))
        Vmean.append(np.mean(Vertical))

        # Median
        Hmedian.append(np.median(abs(Horizontal)))
        Vmedian.append(np.median(abs(Vertical)))

        # std
        Hstd.append(np.std(Horizontal))
        Vstd.append(np.std(Vertical))

        # Skewness
        Hskewness.append(stats.skew(Horizontal))
        Vskewness.append(stats.skew(Vertical))

        # Crest
        Hcrest.append(
            np.max(np.abs(Horizontal)) / np.sqrt(np.mean(np.square(Horizontal)))
        )
        Vcrest.append(np.max(np.abs(Vertical)) / np.sqrt(np.mean(np.square(Vertical))))

        # Energy
        Henergy.append(np.sum(np.abs(Horizontal) ** 2))
        Venergy.append(np.sum(np.abs(Vertical) ** 2))

        # Shapiro
        Hs, Hp = stats.shapiro(Horizontal)
        Hshapiro.append(Hs)
        Vs, Vp = stats.shapiro(Vertical)
        Vshapiro.append(Vs)

        # KL
        x = np.linspace(min(Horizontal), max(Horizontal), 100)
        Hen = stats.entropy(
            stats.gaussian_kde(Horizontal).evaluate(x),
            stats.norm.pdf(x, np.mean(Horizontal), np.std(Horizontal)),
        )
        Hkl.append(Hen)
        x = np.linspace(min(Vertical), max(Vertical), 100)
        Ven = stats.entropy(
            stats.gaussian_kde(Vertical).evaluate(x),
            stats.norm.pdf(x, np.mean(Vertical), np.std(Vertical)),
        )
        Vkl.append(Ven)

        # Reverse KL
        x = np.linspace(min(Horizontal), max(Horizontal), 100)
        Hren = stats.entropy(
            stats.norm.pdf(x, np.mean(Horizontal), np.std(Horizontal)),
            stats.gaussian_kde(Horizontal).evaluate(x),
        )
        Hrkl.append(Hren)
        x = np.linspace(min(Vertical), max(Vertical), 100)
        Vren = stats.entropy(
            stats.norm.pdf(x, np.mean(Vertical), np.std(Vertical)),
            stats.gaussian_kde(Vertical).evaluate(x),
        )
        Vrkl.append(Vren)

    df = pd.DataFrame(Hzerocross, columns=["Hzerocross"])
    df["Hkurtosis"] = Hkurtosis
    df["Hrms"] = Hrms
    df["Hpeaks"] = Hpeaks
    df["Hmean"] = Hmean
    df["Hstd"] = Hstd
    df["Hmedian"] = Hmedian
    df["Hskewness"] = Hskewness
    df["Hcrest"] = Hcrest
    df["Henergy"] = Henergy
    df["Hshapiro"] = Hshapiro
    df["Hkl"] = Hkl
    df["Hrkl"] = Hrkl
    df["Vzerocross"] = Vzerocross
    df["Vkurtosis"] = Vkurtosis
    df["Vrms"] = Vrms
    df["Vpeaks"] = Vpeaks
    df["Vmean"] = Vmean
    df["Vstd"] = Vstd
    df["Vmedian"] = Vmedian
    df["Vskewness"] = Vskewness
    df["Vcrest"] = Vcrest
    df["Venergy"] = Venergy
    df["Vshapiro"] = Vshapiro
    df["Vkl"] = Vkl
    df["Vrkl"] = Vrkl

    mask = df["Hkl"] != np.inf
    df.loc[~mask, "Hkl"] = df.loc[mask, "Hkl"].max()

    mask = df["Vkl"] != np.inf
    df.loc[~mask, "Vkl"] = df.loc[mask, "Vkl"].max()

    mask = df["Hrkl"] != np.inf
    df.loc[~mask, "Hrkl"] = df.loc[mask, "Hrkl"].max()

    mask = df["Vrkl"] != np.inf
    df.loc[~mask, "Vrkl"] = df.loc[mask, "Vrkl"].max()

    return df

X = np.empty((0, 26), float)
rootdir = "/content/merged_files/"
column_headers = ["Hzerocross", "Hkurtosis", "Hrms", "Hpeaks", "Hmean", "Hstd", "Hmedian", "Hskewness",
                  "Hcrest", "Henergy", "Hshapiro", "Hkl", "Hrkl", "Vzerocross", "Vkurtosis", "Vrms",
                  "Vpeaks", "Vmean", "Vstd", "Vmedian", "Vskewness", "Vcrest", "Venergy", "Vshapiro",
                  "Vkl", "Vrkl"]

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        print(file)
        df = pd.read_csv(rootdir + "/" + file)
        features = ToTime(df)
        X = np.append(X, features.values, axis=0)

pd.DataFrame(X, columns=column_headers).to_csv("/content/Xtrain_time.csv", index=False)
df_time = pd.DataFrame(X, columns=column_headers)

Bearing1_6.csv
Bearing2_4.csv
Bearing2_3.csv
Bearing1_5.csv
Bearing3_3.csv
Bearing1_4.csv
Bearing1_3.csv
Bearing1_7.csv
Bearing2_5.csv
Bearing2_6.csv
Bearing2_7.csv


In [ ]:
split_sizes = [321, 320, 321, 320]
start_idx = 0
split_dfs_freq = []
for size in split_sizes:
    split_df = df_freq.iloc[:, start_idx:start_idx + size]
    split_dfs_freq.append(split_df)
    start_idx += size

# Access each split DataFrame separately
for i, split_df in enumerate(split_dfs_freq):
    print(f"Split DataFrame {i+1} shape:", split_df.shape)

frequency_fg_list = []
for i, split_df in enumerate(split_dfs_freq):
    split_df = split_df.reset_index()
    frequency_fg = fs.get_or_create_feature_group(
      name= f"frequency_domain_features_{i}",
      version=1,
      description="vertical and horizontal vibration signals split into 4 feature groups",
      online_enabled=True,
      primary_key=["index"]
    )
    frequency_fg.insert(split_df)
    frequency_fg_list.append(frequency_fg)

Split DataFrame 1 shape: (13959, 321)
Split DataFrame 2 shape: (13959, 320)
Split DataFrame 3 shape: (13959, 321)
Split DataFrame 4 shape: (13959, 320)


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/895126/fs/889925/fg/1042762


Uploading Dataframe: 0.00% |          | Rows 0/13959 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: frequency_domain_features_0_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/895126/jobs/named/frequency_domain_features_0_1_offline_fg_materialization/executions


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/895126/fs/889925/fg/1040726


...

Uploading Dataframe: 0.00% |          | Rows 0/13959 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: frequency_domain_features_2_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/895126/jobs/named/frequency_domain_features_2_1_offline_fg_materialization/executions


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/895126/fs/889925/fg/1041749


Uploading Dataframe: 0.00% |          | Rows 0/13959 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: frequency_domain_features_3_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/895126/jobs/named/frequency_domain_features_3_1_offline_fg_materialization/executions


In [ ]:
from autoencoder import get_AElabels
rootdir = "/content/merged_files/"

Y = []

def ToFrequency_labels(df):
    X = np.empty((0, 1282), float)

    for i in range(0, len(df), 2560):
        Hshort = signal.decimate(df["Hacc"][i : i + 2560], 2)
        Vshort = signal.decimate(df["Vacc"][i : i + 2560], 2)
        Hf = np.abs(rfft(Hshort))
        Vf = np.abs(rfft(Vshort))
        x = np.append(Hf, Vf)
        X = np.append(X, np.array([x]), axis=0)

    return X

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        print(file)
        df = pd.read_csv(rootdir + "/" + file)
        X = ToFrequency_labels(df)
        X_truncated = X[: int(len(X) * 0.8), :]
        labels = get_AElabels(X_truncated, X)
        Y.extend(labels.tolist())

df_labels = pd.DataFrame(Y, columns=["labels"])

Bearing1_6.csv
58/58 [==============================] - 0s 2ms/step


Bearing2_4.csv
16/16 [==============================] - 0s 2ms/step


Bearing2_3.csv
31/31 [==============================] - 0s 2ms/step


Bearing1_5.csv
58/58 [==============================] - 0s 2ms/step


Bearing3_3.csv
9/9 [==============================] - 0s 2ms/step


...

Bearing1_3.csv
46/46 [==============================] - 0s 4ms/step


Bearing1_7.csv
38/38 [==============================] - 0s 4ms/step


Bearing2_5.csv
51/51 [==============================] - 0s 2ms/step


Bearing2_6.csv
15/15 [==============================] - 0s 3ms/step


Bearing2_7.csv
5/5 [==============================] - 0s 9ms/step


In [ ]:
df_with_labels = pd.concat([df_time, df_labels], axis=1)
df_with_labels = df_with_labels.reset_index()
time_fg = fs.get_or_create_feature_group(
    name="time_domain_features",
    version=1,
    description="Time based features",
    online_enabled=True,
    primary_key=["index"]
)
time_fg.insert(df_with_labels)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/895126/fs/889925/fg/1041750


Uploading Dataframe: 0.00% |          | Rows 0/13959 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_domain_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/895126/jobs/named/time_domain_features_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7bbd7a6eb730>, None)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from random import randint
from NNclassifier import create_model
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

In [ ]:
query = (frequency_fg_list[0].select_all()
    .join(frequency_fg_list[1].select_all(), on="index")
    .join(frequency_fg_list[2].select_all(), on="index")
    .join(frequency_fg_list[3].select_all(), on="index")
    .join(time_fg.select_all(), on="index"))

feature_view = fs.get_or_create_feature_view(
    name="bearing_monitoring",
    version=1,
    query=query,
    labels=["labels"])

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/895126/fs/889925/fv/bearing_monitoring/version/1


In [ ]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(
    test_size=0.1,
    description='Bearing monitor',
    primary_keys=False
)

X_train.drop(columns=["index"])
# Extract horizontal data
horizontal_data = X_train.iloc[:, :641].values

# Extract vertical data
vertical_data = X_train.iloc[:, 641:1282].values  # Assuming the next 641 columns after horizontal

# Extract meta data
meta_data = X_train.iloc[:, -26:].values

y_train_array = keras.utils.to_categorical(y_train.values, num_classes=4)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (50.51s) 


In [ ]:
AEclassifier = create_model()
history = AEclassifier.fit(
     [horizontal_data, vertical_data, meta_data], y_train_array, batch_size=512, epochs=15, shuffle=True
)

Epoch 1/15
25/25 [==============================] - 9s 64ms/step - loss: 8.5487 - accuracy: 0.5753
Epoch 2/15
25/25 [==============================] - 1s 29ms/step - loss: 4.1850 - accuracy: 0.6026
Epoch 3/15
25/25 [==============================] - 1s 29ms/step - loss: 2.2316 - accuracy: 0.6389
Epoch 4/15
25/25 [==============================] - 1s 29ms/step - loss: 1.3416 - accuracy: 0.6990
Epoch 5/15
25/25 [==============================] - 1s 28ms/step - loss: 0.9539 - accuracy: 0.7183
Epoch 6/15
25/25 [==============================] - 1s 29ms/step - loss: 0.7728 - accuracy: 0.7377
Epoch 7/15
25/25 [==============================] - 1s 28ms/step - loss: 0.6828 - accuracy: 0.7449
Epoch 8/15
25/25 [==============================] - 1s 29ms/step - loss: 0.6401 - accuracy: 0.7517
Epoch 9/15
25/25 [==============================] - 1s 29ms/step - loss: 0.5971 - accuracy: 0.7678
Epoch 10/15
25/25 [==============================] - 1s 27ms/step - loss: 0.5800 - accuracy: 0.7729
Epoch 11/

In [ ]:
mr = project.get_model_registry()

dtype = [('horizontal', np.float32, (horizontal_data.shape[1],)),
         ('vertical', np.float32, (vertical_data.shape[1],)),
         ('meta', np.float32, (meta_data.shape[1],))]
combined_array = np.zeros(horizontal_data.shape[0], dtype=dtype)

combined_array['horizontal'] = horizontal_data
combined_array['vertical'] = vertical_data
combined_array['meta'] = meta_data

input_schema = Schema(combined_array)
output_schema = Schema(y_train_array)
model_schema = ModelSchema(
    input_schema=input_schema,
    output_schema=output_schema,
)

model_schema.to_dict()

model_dir = "bearing_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

AEclassifier.save(model_dir + '/' + 'AE_classifier.pkl')
metrics = {key: value[-1] for key, value in history.history.items()}

model_entry = mr.tensorflow.create_model(
    name=model_dir,
    description="new model",
    version=1,
    model_schema=model_schema,
    metrics=metrics)


Connected. Call `.close()` to terminate connection gracefully.


In [ ]:
model_entry.save(model_dir)

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/59 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/410114 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/43076 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/5701 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/5049906 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/292 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/895126/models/bearing_model/1


Model(name: 'bearing_model', version: 1)